In [17]:
import tensorflow as tf

In [18]:
import pandas as pd
from collections import Counter

In [65]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import os
import random
import numpy as np
import pickle
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
from ReLERNN import *

# Function to get the Target from the simulation file

In [21]:

def normalizeTargets(i,file):
    infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+file,"info.p")
    nTargets_1 = pickle.load(open(infoFilename,"rb"))['rho']

    norm = 'zscore'
    if(norm == 'zscore'):
        tar_mean = np.mean(nTargets_1,axis=0)
        tar_sd = np.std(nTargets_1,axis=0)
        nTargets =nTargets_1- tar_mean
        nTargets = np.divide(nTargets,tar_sd,out=np.zeros_like(nTargets),where=tar_sd!=0)

    return nTargets[i]

# Calculate the score based on z-score

In [22]:
def z_normalizeTargets(error,file):
    infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+file+'/',"info.p")
    nTargets_1 = pickle.load(open(infoFilename,"rb"))['rho']

    tar_mean = np.mean(nTargets_1,axis=0)
    tar_sd = np.std(nTargets_1,axis=0)
    erro = np.dot(error,tar_sd)+tar_mean


    return erro

# Function to Bin the Minor Allele Frequency

In [23]:
def SNP_WINDOW(w,r,i,file):
 
    respectiveNormalizedTargets = [normalizeTargets(i,file)]
    targets = np.array(respectiveNormalizedTargets)
    lis=[]

    for i in w:
        co = dict(Counter(i))
        if len(co.keys())==2:
            lis.append(co[min(co,key=co.get)]/len(i))
    bins = np.linspace(0, 1, 20)

    bin_means,bins = np.histogram(lis, bins, normed=True, density=True)
    bins= bins[1:]
    bin_means= bin_means/bin_means.sum()
    

    
    

    return np.array(bins),np.array(targets),np.array(bin_means)




# Get the Values and Create batch

In [24]:


def __data_generation1(batchTreeIndices,val):

        haps = []
        pos = []
        for treeIndex in batchTreeIndices:
            Hfilepath = os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+val+'/val/',str(treeIndex) + "_haps.npy")
            Pfilepath = os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+val+'/val/',str(treeIndex) + "_pos.npy")
            H = np.load(Hfilepath)
            P = np.load(Pfilepath)
            haps.append(H)
            pos.append(P)

       
        return haps,pos


            
def __getitem__1(idx,val):
    X,x1= __data_generation1([idx],val)
    return X,x1


# Create the Model

In [25]:
os.environ['PYTHONHASHSEED']=str(42)
random.seed(42)
np.random.seed(42)
#nProc = mp.cpu_count()
   

projectDir = '/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool'
trainDir = os.path.join(projectDir,"train")
valiDir = os.path.join(projectDir,"vali")
testDir = os.path.join(projectDir,"test")
networkDir = os.path.join(projectDir,"networks")


test_resultFile = os.path.join(networkDir,"testResults.p")
test_resultFig = os.path.join(networkDir,"testResults.pdf")
modelSave = os.path.join(networkDir,"model.json")
weightsSave = os.path.join(networkDir,"weights.h5")


maxSimS = 0
winFILE=os.path.join(networkDir,"windowSizes.txt")
with open(winFILE, "r") as fIN:
    for line in fIN:
        maxSimS=max([maxSimS, int(line.split()[5])])
maxSegSites = 0

for ds in [trainDir,valiDir,testDir]:
    DsInfoDir = pickle.load(open(os.path.join(ds,"info.p"),"rb"))
    segSitesInDs = max(DsInfoDir["segSites"])
    maxSegSites = max(maxSegSites,segSitesInDs)
maxSegSites = max(maxSegSites, maxSimS)





x,pos= __getitem__1(0,'train')


vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0],0,'train')

value = tf.convert_to_tensor([vale1],dtype=None)

xTrain = np.reshape(value[0], (value.shape[0], value.shape[1],1))

sites= xTrain.shape

genotype_inputs = layers.Input(shape=(xTrain.shape[1],xTrain.shape[2]))


model1 = layers.Bidirectional(layers.GRU(84,return_sequences=False))(genotype_inputs)
model1 = layers.Dense(256)(model1)
model1 = layers.Dropout(0.35)(model1)
output = layers.Dense(1)(model1)

model1 = Model(inputs=genotype_inputs, outputs=[output])
model1.compile(optimizer='Adam', loss='mse')
model1.summary()





/tmp/ipykernel_30972/923325695.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 19, 1)]           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 168)              43848     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 256)               43264     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 87,369
Trainable params: 87,369
Non-trainable params: 0
_______________________________________________________

# Create Batch file for Training Set

In [55]:
lis_=[]
train_=[]
for i in range(13000):
    x,pos= __getitem__1(i,'train')
    vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0],i,'train')

    lis_.append(vale1)
    train_.append(targets)
    
value_train = tf.convert_to_tensor(lis_,dtype=None)
target_train = tf.convert_to_tensor(train_,dtype=None)


/tmp/ipykernel_30972/923325695.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


# Create Batch for Testing Data

In [27]:
lis_=[]
train_=[]
for i in range(100):
    x,pos= __getitem__1(i,'test')
    vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0],i,'test')

    lis_.append(vale1)
    train_.append(targets)
    
value_test = tf.convert_to_tensor(lis_,dtype=None)
target_test = tf.convert_to_tensor(train_,dtype=None)


/tmp/ipykernel_30972/923325695.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


# Create Batch for validation Data

In [28]:
vali_x=[]
vali_y=[]
for i in range(1000):
    x,pos= __getitem__1(i,'vali')

    
    vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0],i,'vali')
    
    vali_x.append(vale1)
    vali_y.append(targets)

value_vali = tf.convert_to_tensor(vali_x,dtype=None)
target_vali = tf.convert_to_tensor(vali_y,dtype=None)


/tmp/ipykernel_30972/923325695.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


In [ ]:
value_vali

# Call Back function

In [29]:
callbacks_list = [
    EarlyStopping(
    monitor='val_loss',
    verbose=1,
    min_delta=0.01,
    patience=100),
    ModelCheckpoint(
    filepath='/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val',
    monitor='val_loss',
    save_best_only=True),
    TerminateOnNaN()]




# Model Fitting

In [47]:
history = model1.fit(value_train,target_train,
steps_per_epoch=1000,
epochs=15,
callbacks=callbacks_list,
validation_data =(value_vali,target_vali),
use_multiprocessing=True)
#model1.fit(xTrain,targets,epochs=5)
    
    
model_json = model1.to_json()
with open('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/model', "w+") as json_file:
    json_file.write(model_json)


history.history['loss'] = np.array(history.history['loss'])
#history.history['val_loss'] = np.array(history.history['val_loss'])
#history.history['predictions'] = np.array(predictions)
history.history['Y_test'] = np.array(targets)
history.history['name'] = 'ModelName'

pickle.dump(history.history, open( 'resultsFile', "wb" ))

Epoch 1/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3767 - val_loss: 0.3783
Epoch 2/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3653 - val_loss: 0.3726
Epoch 3/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3654 - val_loss: 0.3501
Epoch 4/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3631 - val_loss: 0.3528
Epoch 5/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3604 - val_loss: 0.3667
Epoch 6/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3609 - val_loss: 0.3511
Epoch 7/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3607 - val_loss: 0.3572
Epoch 8/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3603 - val_loss: 0.3390
Epoch 9/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3572 - val_loss: 0.3393
Epoch 10/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.362

INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


1000/1000 [==============================] - 24s 24ms/step - loss: 0.3523 - val_loss: 0.3346
Epoch 15/15
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3522 - val_loss: 0.3380


In [61]:
infoFilename= os.path.join("resultsFile")
ne = pickle.load(open(infoFilename,"rb"))
ne['val_loss']

[0.3782947361469269,
 0.3726363182067871,
 0.3501492142677307,
 0.3528112471103668,
 0.36674559116363525,
 0.3510673940181732,
 0.3571929335594177,
 0.33895668387413025,
 0.33926841616630554,
 0.3385832905769348,
 0.343308687210083,
 0.3430556356906891,
 0.34782299399375916,
 0.3345509469509125,
 0.33798807859420776]

In [68]:
de2= pd.DataFrame([ne['val_loss'],list(range(15))]).T

,0,1
0,0.378295,0.0
1,0.372636,1.0
2,0.350149,2.0
3,0.352811,3.0
4,0.366746,4.0
5,0.351067,5.0
6,0.357193,6.0
7,0.338957,7.0
8,0.339268,8.0
9,0.338583,9.0


In [72]:

plt.clf()
ax =sns.lineplot(data=de2,x=1,y=0)
ax.set(xlabel='Val_loss',ylabel='Epoch')
plt.savefig("/N/u/samishr/Carbonate/Desktop/Recombination_landscape/Data_Processing/epoch_loss.png")
plt.clf()

In [78]:
erro=model1.predict(value_test)-target_test

In [79]:
erro= z_normalizeTargets(erro,'test')

In [80]:
de1= pd.DataFrame(erro)

In [81]:
de1

,0
0,5.476100e-09
1,6.305009e-09
2,3.490014e-09
3,4.929563e-09
4,3.922370e-09
...,...
95,4.158451e-09
96,5.779596e-09
97,5.434546e-09
98,3.735014e-09


In [82]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.clf()
ax = sns.boxplot(data=de1)
ax.set(xlabel='True',ylabel='Epoch')
plt.savefig("/N/u/samishr/Carbonate/Desktop/Recombination_landscape/Data_Processing/fig1.png")

# Done

In [ ]:
infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/',"info.p")
ne = pickle.load(open(infoFilename,"rb"))['Ne']

In [ ]:
nTargets_1

In [ ]:
mu = pickle.load(open(infoFilename,"rb"))['mu']

In [ ]:
nTargets_1